In [1]:
import pandas as pd
import numpy as np

In [2]:
bot_posts_df = pd.read_excel('./training_data/bot_app_posts.xls')
posts_df = pd.read_excel('./training_data/posts.xlsx')

In [8]:
posts_df.dtypes

id             object
telegram_id    object
text           object
datetime       object
category_id    object
channel_id     object
title          object
Unnamed: 7     object
Unnamed: 8     object
Unnamed: 9     object
dtype: object

In [9]:
bot_posts_df.dtypes

user_id         int64
telegram_id     int64
text           object
title          object
dtype: object

In [3]:
bot_posts_df.head()

,user_id,telegram_id,text,title
0,1,157314,❗️Восстановление аммиакопровода Тольятти — Оде...,Восстановление аммиакопровода займет до 3 меся...
1,2,18940,Дополнительные 39 миллионов рублей были выделе...,Выделены дополнительные 39 млн рублей на соцуч...
2,3,4868,«Россия уничтожена санкциями»\n ...,«Россия уничтожена санкциями»\n ...
3,4,157315,❗️Правоохранители провели обыски в министерств...,Обыски в минобразования Дагестана по делу о вы...
4,16,157317,"Реконструкция пропускного пункта ""Верхний Ларс...","Реконструкция пропускного пункта ""Верхний Ларс..."


In [25]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer

class textPreprocesser:

  def __init__(self, df: pd.DataFrame, columns_for_preprocessing: list) -> None:
    nltk.download('punkt')
    nltk.download('stopwords')

    self.stemmer = SnowballStemmer('russian')
    self.stop_words = set(
      stopwords.words('russian') + stopwords.words('english')
    )
    self.cols = columns_for_preprocessing
    self.df = df

  def tokenize(self, token_string: str) -> list:
    return [w for s in sent_tokenize(token_string) for w in word_tokenize(s)]

  def stem(self, tokens: list) -> list:
    return [self.stemmer.stem(token) for token in tokens]

  def clean(self) -> None:
    for column in self.cols:
      self.df[column] = self.df[column].str.lower()
      self.df[column] = self.df[column].str.replace('[^0-9A-Za-zА-Яа-я.\s]+', '', regex=True)
      self.df[column] = self.df[column].str.replace('quot','"')
      self.df[column] = self.df[column].str.replace('|'.join([x + '\s' for x in self.stop_words]), '', regex=True)
      self.df[column] = self.df[column].apply(self.tokenize)
      self.df[column] = self.df[column].apply(self.stem)

In [29]:
preprocesserClass = textPreprocesser(bot_posts_df[~bot_posts_df['text'].isna()], ['text'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
preprocesserClass.clean()

In [31]:
preprocesserClass.df.head()

,user_id,telegram_id,text,title
0,1,157314,"[восстановлен, аммиакопровотольятт, одессзайме...",восстановление аммиакопровода займет до 3 меся...
1,2,18940,"[дополнительн, 39, миллионорублвыдел, социальн...",выделены дополнительные 39 млн рублей на соцуч...
2,3,4868,"[россиуничтожесанкц, байд, ., ахх, httpst.mert...",россия уничтожена санкциями\n ...
3,4,157315,"[правоохранитепровеобыскминистерств, образован...",обыски в минобразования дагестана по делу о вы...
4,16,157317,"[реконструкципропускногпунктверхн, ларзавершит...",реконструкция пропускного пункта верхний ларс ...
